In [1]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
import urllib

In [2]:
%load_ext dotenv
%dotenv
load_dotenv("BEERMAPPING_KEY")

True

### Perform at least 5 data transformation and/or cleansing steps to your website data.
For Example:
* Replace Headers
* Format data into a more readable format
* Identify outliers and bad data
* Find duplicates
* Fix casing or inconsistent values
* Conduct Fuzzy Matching

In [3]:
apiKey = os.environ.get("BEERMAPPING_KEY")

In [4]:
baseURL = "http://beermapping.com/webservice/"

https://beermapping.com/api/reference/

Basic URL format: http://beermapping.com/webservice/{SERVICE}/API_KEY/{QUERY}  
Append '&s=json' to return JSON instead of XML

In [5]:
# # Search by City,State
# service = "loccity"

In [7]:
# Search by State
service = "locstate"
query = str(input("Enter 2-letter US State acronym:"))

Enter 2-letter US State acronym: ne


In [9]:
requestURL = f"{baseURL}{service}/{apiKey}/{query}&s=json"

In [11]:
response = requests.request("GET", requestURL)
print(response.status_code)
if response.status_code != 200:
    raise Exception(response.status_code, response.text)
jsonResponse = response.json()

200


In [13]:
type(jsonResponse)

list

In [34]:
# jsonResponse

In [137]:
jsonResponse['numberOfPages']

1

In [14]:
len(jsonResponse)

105

In [15]:
jsonResponse[0]

{'id': 21938,
 'name': '5168 Brewing',
 'status': 'Brewery',
 'reviewlink': 'https://beermapping.com/location/21938',
 'proxylink': 'http://beermapping.com/maps/proxymaps.php?locid=21938&amp;d=5',
 'blogmap': 'http://beermapping.com/maps/blogproxy.php?locid=21938&amp;d=1&amp;type=norm',
 'street': '5730 Hidcote Drive',
 'city': 'Lincoln',
 'state': 'NE',
 'zip': '68516',
 'country': 'United States',
 'phone': '402-875-5588',
 'url': '',
 'overall': '0',
 'imagecount': '0'}

In [30]:
jsonResponse[-1]

{'id': 21370,
 'name': 'Zipline Brewing Company',
 'status': 'Brewery',
 'reviewlink': 'https://beermapping.com/location/21370',
 'proxylink': 'http://beermapping.com/maps/proxymaps.php?locid=21370&amp;d=5',
 'blogmap': 'http://beermapping.com/maps/blogproxy.php?locid=21370&amp;d=1&amp;type=norm',
 'street': '721 N 14th St',
 'city': 'Omaha',
 'state': 'NE',
 'zip': '68102',
 'country': 'United States',
 'phone': '',
 'url': 'http%3A%2F%2Fziplinebrewing.com%2F',
 'overall': '0',
 'imagecount': '0'}

In [32]:
jsonResponse[0].keys()

dict_keys(['id', 'name', 'status', 'reviewlink', 'proxylink', 'blogmap', 'street', 'city', 'state', 'zip', 'country', 'phone', 'url', 'overall', 'imagecount'])

In [36]:
def getStateBreweries(state):
    '''
    Function to retrieve data about breweries within a US State
    '''
    # Assign URL Service to search by State:
    service = "locstate"
    # Assign search term:
    query = state
    # Assemble URL:
    requestURL = f"{baseURL}{service}/{apiKey}/{query}&s=json"
        # Note '&s=json' appended to return JSON formatted data
    
    response = requests.request("GET", requestURL)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    jsonResponse = response.json()
    
    return jsonResponse

In [37]:
state = str(input("Enter 2-letter US State acronym:"))
breweryJSON = getStateBreweries(state)

Enter 2-letter US State acronym: NE


200


In [38]:
breweryJSON[0].keys()

dict_keys(['id', 'name', 'status', 'reviewlink', 'proxylink', 'blogmap', 'street', 'city', 'state', 'zip', 'country', 'phone', 'url', 'overall', 'imagecount'])

In [76]:
def buildBreweryDB(listBreweries):
    '''
    Take a list of Brewery data in JSON format.
    Output a DF with key information about those breweries.
    '''
    # Define an empty dictionary, with keys:
    breweryDict = {'BreweryID': [], 'BreweryName': [], 'Type': [], 'ReviewLink': [],
                   'ProxyLink': [], 'Map': [], 'StreetAddress': [], 'City': [],
                   'State': [], 'Zip': [], 'Country': [], 'PhoneNum': [],
                   'Website': [], 'Rating': [], 'ImageCount': []}
    
    for b in listBreweries:
        breweryDict['BreweryID'].append(b['id'])
        breweryDict['BreweryName'].append(b['name'])
        breweryDict['Type'].append(b['status'])
        breweryDict['ReviewLink'].append(b['reviewlink'])
        breweryDict['ProxyLink'].append(b['proxylink'])
        breweryDict['Map'].append(b['blogmap'])
        breweryDict['StreetAddress'].append(b['street'])
        breweryDict['City'].append(b['city'])
        breweryDict['State'].append(b['state'])
        breweryDict['Zip'].append(b['zip'])
        breweryDict['Country'].append(b['country'])
        breweryDict['PhoneNum'].append(b['phone'])
#         breweryDict['Website'].append(b['url'])
        breweryDict['Website'].append(urllib.parse.unquote(b['url'])) # Fix parsed URLs
        breweryDict['Rating'].append(b['overall'])
        breweryDict['ImageCount'].append(b['imagecount'])
        
    return pd.DataFrame(breweryDict)

In [73]:
urllib.parse.unquote(test)

'http://www.5168brewing.com/'

In [122]:
breweryDF = buildBreweryDB(breweryJSON)

In [119]:
breweryDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
1,22335,5168 Taproom,Beer Bar,https://beermapping.com/location/22335,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3201 Farnam Street,Omaha,NE,68131,United States,402-934-5168,http://www.5168brewing.com/,0,0
2,21215,Alamo Draft House,Beer Bar,https://beermapping.com/location/21215,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,12750 Westport Pkwy,La Vista,NE,68138,United States,402-505-9979,https://drafthouse.com/omaha,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0


### Issue with parsed URLs being returned:

In [57]:
test = breweryDF.loc[1][12]
test

'http%3A%2F%2Fwww.5168brewing.com%2F'

In [58]:
from requests.utils import requote_uri

In [59]:
requote_uri(test)

'http%3A%2F%2Fwww.5168brewing.com%2F'

In [61]:
test.decode('UTF-8')

AttributeError: 'str' object has no attribute 'decode'

In [63]:
from urllib.parse import urlparse

In [69]:
from urllib.parse import urlunparse

In [65]:
import urllib

In [66]:
urllib.parse.quote(test)

'http%253A%252F%252Fwww.5168brewing.com%252F'

In [70]:
urllib.parse.urlunparse(test)

ValueError: too many values to unpack (expected 7)

In [72]:
urllib.unquote(test.encode("utf8"))

AttributeError: module 'urllib' has no attribute 'unquote'

In [73]:
urllib.parse.unquote(test)

'http://www.5168brewing.com/'

We have a winner!!

In [75]:
# Probably just this?
import urllib

In [45]:
# breweryDict = {'BreweryID': [], 'BreweryName': [], 'Type': [], 'ReviewLink': [],
#                    'ProxyLink': [], 'Map': [], 'StreetAddress': [], 'City': [],
#                    'State': [], 'Zip': [], 'Country': [], 'PhoneNum': [],
#                    'Website': [], 'Overall': [], 'ImageCount': []}

In [43]:
# breweryDict = {'id': [], 'name': [], 'status': [], 'reviewlink': [],
#                    'proxylink': [], 'blogmap': [], 'street': [], 'city': [],
#                    'state': [], 'zip': [], 'country': [], 'phone': [],
#                    'url': [], 'overall': [], 'imagecount': []}

In [46]:
# for b in breweryJSON:
#     breweryDict['BreweryID'].append(b['id'])
#     breweryDict['BreweryName'].append(b['name'])
#     breweryDict['Type'].append(b['status'])
#     breweryDict['ReviewLink'].append(b['reviewlink'])
#     breweryDict['ProxyLink'].append(b['proxylink'])
#     breweryDict['Map'].append(b['blogmap'])
#     breweryDict['StreetAddress'].append(b['street'])
#     breweryDict['City'].append(b['city'])
#     breweryDict['State'].append(b['state'])
#     breweryDict['Zip'].append(b['zip'])
#     breweryDict['Country'].append(b['country'])
#     breweryDict['PhoneNum'].append(b['phone'])
#     breweryDict['Website'].append(b['url'])
#     breweryDict['Rating'].append(b['overall'])
#     breweryDict['ImageCount'].append(b['imagecount'])

In [50]:
# breweryDict

In [48]:
# df = pd.DataFrame(breweryDict)

In [49]:
# df.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Overall,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
1,22335,5168 Taproom,Beer Bar,https://beermapping.com/location/22335,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3201 Farnam Street,Omaha,NE,68131,United States,402-934-5168,http%3A%2F%2Fwww.5168brewing.com%2F,0,0
2,21215,Alamo Draft House,Beer Bar,https://beermapping.com/location/21215,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,12750 Westport Pkwy,La Vista,NE,68138,United States,402-505-9979,https%3A%2F%2Fdrafthouse.com%2Fomaha,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com%2FBackswingBrewingCo%2Ftimeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0


In [18]:
# breweriesStr = json.dumps(jsonResponse, indent=4, sort_keys=True)

In [25]:
# type(breweriesStr)

In [26]:
# breweriesStr[:500]

In [22]:
# breweriesJSON = json.loads(breweriesStr)

In [27]:
# type(breweriesJSON)

In [29]:
# breweriesJSON[0]

## DataFrame time:

In [79]:
breweryDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
1,22335,5168 Taproom,Beer Bar,https://beermapping.com/location/22335,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3201 Farnam Street,Omaha,NE,68131,United States,402-934-5168,http://www.5168brewing.com/,0,0
2,21215,Alamo Draft House,Beer Bar,https://beermapping.com/location/21215,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,12750 Westport Pkwy,La Vista,NE,68138,United States,402-505-9979,https://drafthouse.com/omaha,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0


In [80]:
breweryDF.Type.unique()

array(['Brewery', 'Beer Bar', 'Beer Store', 'Brewpub', 'Homebrew'],
      dtype=object)

Save a backup, in case the site goes down again:

In [81]:
breweryDF.to_csv('beerMapping.csv', index=False)

In [82]:
breweryDF[breweryDF['Type'] == 'Beer Bar']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
1,22335,5168 Taproom,Beer Bar,https://beermapping.com/location/22335,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3201 Farnam Street,Omaha,NE,68131,United States,402-934-5168,http://www.5168brewing.com/,0,0
2,21215,Alamo Draft House,Beer Bar,https://beermapping.com/location/21215,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,12750 Westport Pkwy,La Vista,NE,68138,United States,402-505-9979,https://drafthouse.com/omaha,0,0
6,15824,Beer Corner USA,Beer Bar,https://beermapping.com/location/15824,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3578 Farnam Street,Omaha,NE,68131,United States,(402) 345-1708,beercornerusa.com,0,0
7,15395,Beercade,Beer Bar,https://beermapping.com/location/15395,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6102 Maple St,Omaha,NE,68102,United States,(402) 934-9633,thebeercade.com,0,0
10,21209,Blat Beer and Table,Beer Bar,https://beermapping.com/location/21209,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,2835 S 170th St #200,Omaha,NE,68135,United States,402-697-7802,http://blattbeer.com/,0,0
11,21208,Blatt Beer and Table,Beer Bar,https://beermapping.com/location/21208,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,610 North 12th street,Omaha,NE,68102,United States,402-718-8822,http://blattbeer.com/,0,0
12,21210,Blatt Beer and Table,Beer Bar,https://beermapping.com/location/21210,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"Westroads Mall, 10000 California St",Omaha,NE,68114,United States,402-932-9993,http://blattbeer.com/,0,0
19,11763,Brass Monkey Sports Bar,Beer Bar,https://beermapping.com/location/11763,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5611 South 36th Street,Omaha,NE,68107,United States,(402) 991-4920‎,brassmonkey.synthasite.com,0,0
23,18786,Cafe Lobbes,Beer Bar,https://beermapping.com/location/18786,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,Hof 10a,Amersfoort,NE,3811 CJ,Netherlands,033 4610637,cafe-lobbes.nl,0,0
24,287,Chicken Coop Sports Bar and Grill,Beer Bar,https://beermapping.com/location/287,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,120 East Third Street,Grand Island,NE,68801,United States,(308) 398-2500,,0,0


Lazlos Haymarket is the location for Empyrean Brewing Co, so I will recategorize from 'Beer Bar' to 'Brewery' and later fix the name. 

In [87]:
breweryDF.iloc[55]

BreweryID                                                    17041
BreweryName         Lazlo's Brewery and Grill - Historic Haymarket
Type                                                      Beer Bar
ReviewLink                  https://beermapping.com/location/17041
ProxyLink        http://beermapping.com/maps/proxymaps.php?loci...
Map              http://beermapping.com/maps/blogproxy.php?loci...
StreetAddress                                         210 N 7th St
City                                                       Lincoln
State                                                           NE
Zip                                                          68508
Country                                              United States
PhoneNum                                              402-434-5636
Website                                  lazlosbreweryandgrill.com
Rating                                                           0
ImageCount                                                    

In [88]:
breweryDF.iloc[55][2]

'Beer Bar'

In [98]:
# breweryDF.iloc[55][2] = 'Brewery'
# <ipython-input-89-e915dbb331a8>:1: SettingWithCopyWarning: 
# A value is trying to be set on a copy of a slice from a DataFrame

In [93]:
breweryDF['Type'][55]

'Beer Bar'

In [99]:
# breweryDF['Type'][55] = 'Brewery'
# <ipython-input-94-d97fd2034ae0>:1: SettingWithCopyWarning: 
# A value is trying to be set on a copy of a slice from a DataFrame

In [100]:
# breweryDF.set_value('Type', 55, 'Brewery')
# AttributeError: 'DataFrame' object has no attribute 'set_value'
# Looks like 'set_value' has been deprecated :(

In [124]:
# DO NOT INCLUDE THIS
breweryDF.at[55, 'Type'] = 'Brewery'

Winner winner, chicken dinner

Okay, scratch that. Empyrean is listed under Brewpubs.

In [97]:
breweryDF[breweryDF['Type'] == 'Brewery']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount,55
0,21938.0,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0,NaN
3,19644.0,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0,NaN
4,21194.0,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0,NaN
5,14326.0,Beaver View Brewing,Brewery,https://beermapping.com/location/14326,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,524 S. 4th Street,Albion,NE,68620,United States,,,0,0,NaN
13,14814.0,Blue Blood Brewing Company,Brewery,https://beermapping.com/location/14814,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,925 Robber's Cave Rd,Lincoln,NE,68522,United States,(402) 477-2337,bluebloodbrewing.com,0,0,NaN
14,20376.0,Boiler Brewing Company,Brewery,https://beermapping.com/location/20376,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"129 N 10th St, Suite 8",Lincoln,NE,68508,United States,402-261-8775,boilerbrewingcompany.com,0,0,NaN
15,20287.0,Bolo Beer Co,Brewery,https://beermapping.com/location/20287,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,420 East 1st St,Valentine,NE,69201,United States,,facebook.com/BoloBeerCo,0,0,NaN
18,20099.0,Bottle Rocket Brewing Company,Brewery,https://beermapping.com/location/20099,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,230 S. 5th Street,Seward,NE,68434,United States,,bottlerocketbrewing.com,0,0,NaN
20,18888.0,Brickway Brewery and Distillery,Brewery,https://beermapping.com/location/18888,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,1116 Jackson Street,Omaha,NE,68102,United States,(402) 933-2613,http://www.drinkbrickway.com/,93.3334,1,NaN
21,19643.0,Broken Arrow Cellars and Brewery,Brewery,https://beermapping.com/location/19643,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,73892 332nd ave,Imperial,NE,69033,United States,﻿308-882-7772,facebook.com/pages/Broken-Arrow-Cellars/793194...,0,0,NaN


In [101]:
breweryDF.Type.unique()

array(['Brewery', 'Beer Bar', 'Beer Store', 'Brewpub', 'Homebrew', nan],
      dtype=object)

In [102]:
breweryDF[breweryDF['Type'] == 'Beer Store']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount,55
8,5570.0,Beertopia,Beer Store,https://beermapping.com/location/5570,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3570 Farnam Street,Omaha,NE,68131,United States,402-905-2603,,0,0,NaN
45,21212.0,HyVee,Beer Store,https://beermapping.com/location/21212,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,14591 Stony Brook Blvd,Omaha,NE,68137,United States,402-697-0811,https://www.hy-vee.com/stores/detail.aspx?s=118,0,0,NaN
51,15833.0,Jakes Cigars and Spirits,Beer Store,https://beermapping.com/location/15833,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6206 Maple Street,Omaha,NE,68104,United States,(402) 934-9633,jakescigars.com,0,0,NaN
88,15849.0,Spirit World,Beer Store,https://beermapping.com/location/15849,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,7517 Pacific Street,Omaha,NE,68114,United States,(402) 391-8680,spiritworldwine.com,0,0,NaN
100,21211.0,Whole Foods Market,Beer Store,https://beermapping.com/location/21211,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,10020 Regency Cir,Omaha,NE,68114,United States,402-393-1200,http://www.wholefoodsmarket.com/stores/omaha,0,0,NaN


In [101]:
breweryDF.Type.unique()

array(['Brewery', 'Beer Bar', 'Beer Store', 'Brewpub', 'Homebrew', nan],
      dtype=object)

In [103]:
breweryDF[breweryDF['Type'] == 'Brewpub']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount,55
9,17706.0,Benson Brewery,Brewpub,https://beermapping.com/location/17706,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6059 Maple Street,Omaha,NE,68104,United States,(402) 937-1892,bensonbrewery.com,0,0,NaN
16,20835.0,Bootleg Brewers,Brewpub,https://beermapping.com/location/20835,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,45145 829th Rd,Taylor,NE,68879,United States,308-942-3440,bootlegbrewers.com,0,0,NaN
17,21191.0,Bootleg Brewers / Sandhills Brewing Company,Brewpub,https://beermapping.com/location/21191,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,45145 829th Rd,Taylor,NE,68879,United States,308-942-3440,bootlegbrewers.com,0,0,NaN
22,20832.0,Brush Creek Brewing Company,Brewpub,https://beermapping.com/location/20832,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,102 N. Main Street,Atkinson,NE,68713,United States,,facebook.com/BrushCreekBrewingCompany/,0,0,NaN
25,21721.0,Code Beer Company,Brewpub,https://beermapping.com/location/21721,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,200 S Antelope Valley Pkwy,Lincoln,NE,68508,United States,(402) 318-5888,http://www.codebeer.co/,0,0,NaN
32,417.0,Empyrean Ales - Lazlos,Brewpub,https://beermapping.com/location/417,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,729 Q St.,Lincoln,NE,68508,United States,(402) 434-5959,telesis-inc.com/empyrean/,0,6,NaN
33,20101.0,Fairfield Opera House Brewery and Grill,Brewpub,https://beermapping.com/location/20101,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,415 North D Street,Fairfield,NE,68938,United States,402-726-2447,fairfieldoperahouse.com,0,0,NaN
34,19001.0,Farnam House Brewing Company,Brewpub,https://beermapping.com/location/19001,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,3558 Farnam Street,Omaha,NE,68131,United States,,farnamhousebrewing.com,0,0,NaN
36,6413.0,Fireworks Restaurant (Empyrean Ales),Brewpub,https://beermapping.com/location/6413,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5750 South 86th Drive,Lincoln,NE,68526,United States,(402)434-5644,telesis-inc.com/fireworks,0,0,NaN
38,22330.0,Flyover Brewing Company,Brewpub,https://beermapping.com/location/22330,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,1824 Broadway,Scottsbluff,NE,69361,United States,,www.flyoverbrewingcompany.com,0,0,NaN


In [101]:
breweryDF.Type.unique()

array(['Brewery', 'Beer Bar', 'Beer Store', 'Brewpub', 'Homebrew', nan],
      dtype=object)

In [104]:
breweryDF[breweryDF['Type'] == 'Homebrew']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount,55
35,15924.0,Fermenter's Supply and Equipment,Homebrew,https://beermapping.com/location/15924,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,8410 K Plaza Suite # 10,Omaha,NE,68127,United States,(402) 593-9171,fermenterssupply.com,0,0,NaN
80,21203.0,Patriot Homebrew Supply,Homebrew,https://beermapping.com/location/21203,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,2929 N 204th St Ste. 107,Omaha,NE,68022,United States,402-991-6655,http://www.patriothomebrewsupply.com/,0,0,NaN


In [126]:
breweryDF.Type.unique()

array(['Brewery', 'Beer Bar', 'Beer Store', 'Brewpub', 'Homebrew'],
      dtype=object)

In [109]:
import numpy as np

In [114]:
breweryDF[breweryDF['Type'] == ]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [125]:
breweryDF[breweryDF['Type'].isnull()]

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount


In [127]:
breweryDF.columns

Index(['BreweryID', 'BreweryName', 'Type', 'ReviewLink', 'ProxyLink', 'Map',
       'StreetAddress', 'City', 'State', 'Zip', 'Country', 'PhoneNum',
       'Website', 'Rating', 'ImageCount'],
      dtype='object')

In [129]:
breweriesDF = breweryDF[(breweryDF['Type'] == 'Brewery') | (breweryDF['Type'] == 'Brewpub')]

In [130]:
breweriesDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0
5,14326,Beaver View Brewing,Brewery,https://beermapping.com/location/14326,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,524 S. 4th Street,Albion,NE,68620,United States,,,0,0
9,17706,Benson Brewery,Brewpub,https://beermapping.com/location/17706,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6059 Maple Street,Omaha,NE,68104,United States,(402) 937-1892,bensonbrewery.com,0,0


In [ ]:
len(breweryDF)

In [44]:
closed = ['Beaver View Brewing', 'Blue Blood Brewing Company',
         "Misty's Steakhouse And Brewery", 'Spilker Ales']

In [48]:
breweryDF = removeBreweries(breweryDF, 'BreweryName', closed)

In [ ]:
len(breweryDF)

In [131]:
testDF = breweryDF.copy()

In [132]:
nameChanges = {
    '5168 Brewing': 'Catalyst Brewing Company',
    'Broken Arrow Cellars and Brewery': 'Hanging Horseshoe Brewing Company',
    'Empyrean Ales - Lazlos': 'Empyrean Brewing Company',
}

In [134]:
breweryDF = breweryDF[(breweryDF['Type'] == 'Brewery') | (breweryDF['Type'] == 'Brewpub')]

In [137]:
def removeBreweries(df, col, values):
    return df[df[col].isin(values) == False]

#### Remove closed breweries:

In [138]:
len(breweryDF)

55

In [139]:
closed = ['Beaver View Brewing', 'Blue Blood Brewing Company',
         "Misty's Steakhouse And Brewery", 'Spilker Ales']

In [140]:
breweryDF = removeBreweries(breweryDF, 'BreweryName', closed)

In [141]:
len(breweryDF)

51

In [142]:
testDF = breweryDF.copy()

In [143]:
testDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0
9,17706,Benson Brewery,Brewpub,https://beermapping.com/location/17706,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6059 Maple Street,Omaha,NE,68104,United States,(402) 937-1892,bensonbrewery.com,0,0
14,20376,Boiler Brewing Company,Brewery,https://beermapping.com/location/20376,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"129 N 10th St, Suite 8",Lincoln,NE,68508,United States,402-261-8775,boilerbrewingcompany.com,0,0


In [150]:
nameChanges = {
    '5168 Brewing': 'Catalyst Brewing Company',
    'Broken Arrow Cellars and Brewery': 'Hanging Horseshoe Brewing Company',
    'Empyrean Ales - Lazlos': 'Empyrean Brewing Company',
}

In [155]:
# for k, v in nameChanges.items():
#     print(k, v)

5168 Brewing Catalyst Brewing Company
Broken Arrow Cellars and Brewery Hanging Horseshoe Brewing Company
Empyrean Ales - Lazlos Empyrean Brewing Company


I always forget to add `.items()`

In [156]:
for k, v in nameChanges.items():
    testDF['BreweryName'].replace(k, v, inplace=True)

In [157]:
testDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,Catalyst Brewing Company,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0
9,17706,Benson Brewery,Brewpub,https://beermapping.com/location/17706,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6059 Maple Street,Omaha,NE,68104,United States,(402) 937-1892,bensonbrewery.com,0,0
14,20376,Boiler Brewing Company,Brewery,https://beermapping.com/location/20376,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"129 N 10th St, Suite 8",Lincoln,NE,68508,United States,402-261-8775,boilerbrewingcompany.com,0,0


In [158]:
def updateNames(df, col, replaceDict):
    for k, v in replaceDict.items():
        df[col].replace(k, v, inplace=True)

In [159]:
testDF = breweryDF.copy()

In [160]:
testDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,5168 Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0
9,17706,Benson Brewery,Brewpub,https://beermapping.com/location/17706,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6059 Maple Street,Omaha,NE,68104,United States,(402) 937-1892,bensonbrewery.com,0,0
14,20376,Boiler Brewing Company,Brewery,https://beermapping.com/location/20376,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"129 N 10th St, Suite 8",Lincoln,NE,68508,United States,402-261-8775,boilerbrewingcompany.com,0,0


In [161]:
updateNames(testDF, 'BreweryName', nameChanges)

In [162]:
testDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,Catalyst Brewing Company,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0
9,17706,Benson Brewery,Brewpub,https://beermapping.com/location/17706,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6059 Maple Street,Omaha,NE,68104,United States,(402) 937-1892,bensonbrewery.com,0,0
14,20376,Boiler Brewing Company,Brewery,https://beermapping.com/location/20376,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"129 N 10th St, Suite 8",Lincoln,NE,68508,United States,402-261-8775,boilerbrewingcompany.com,0,0


In [147]:
# testDF[testDF['BreweryName'] == '5168 Brewing']['BreweryName'] = 'Catalyst Brewing'

In [146]:
# dfAge[dfAge['Age'] > 122] = np.nan

In [60]:
testDF['BreweryName'].replace('5168 Brewing', 'Catalyst Brewing', inplace=True)

In [61]:
testDF.head()

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
0,21938,Catalyst Brewing,Brewery,https://beermapping.com/location/21938,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,5730 Hidcote Drive,Lincoln,NE,68516,United States,402-875-5588,,0,0
3,19644,Backswing Brewing,Brewery,https://beermapping.com/location/19644,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,(402) 515-4263,facebook.com/BackswingBrewingCo/timeline,0,0
4,21194,Backswing Brewing Company,Brewery,https://beermapping.com/location/21194,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,500 W South St #8,Lincoln,NE,68522,United States,402-515-4263,,0,0
9,17706,Benson Brewery,Brewpub,https://beermapping.com/location/17706,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6059 Maple Street,Omaha,NE,68104,United States,(402) 937-1892,bensonbrewery.com,0,0
14,20376,Boiler Brewing Company,Brewery,https://beermapping.com/location/20376,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"129 N 10th St, Suite 8",Lincoln,NE,68508,United States,402-261-8775,boilerbrewingcompany.com,0,0


In [166]:
testDF[testDF['BreweryName'].duplicated()]

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
47,20515,Infusion Brewing Company,Brewery,https://beermapping.com/location/20515,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6271 South 118th Street,Omaha,NE,68137,United States,402-934-2064,infusionbrewing.com,0,0
69,18889,Nebraska Brewing Company,Brewery,https://beermapping.com/location/18889,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6950 S 108th St,La Vista,NE,68128,United States,,nebraskabrewingco.com,0,0
95,16272,Thunderhead Brewing Company,Brewery,https://beermapping.com/location/16272,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,201 Avenue F,Axtell,NE,68924,United States,(308) 237-1558,thunderheadbrewing.com,0,0
104,21370,Zipline Brewing Company,Brewery,https://beermapping.com/location/21370,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,721 N 14th St,Omaha,NE,68102,United States,,http://ziplinebrewing.com/,0,0


In [167]:
testDF[testDF['BreweryName'] == 'Infusion Brewing Company']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
46,18081,Infusion Brewing Company,Brewery,https://beermapping.com/location/18081,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6115 Maple Street,Omaha,NE,68104,United States,(402) 916-9998,infusionbrewing.com,0,0
47,20515,Infusion Brewing Company,Brewery,https://beermapping.com/location/20515,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6271 South 118th Street,Omaha,NE,68137,United States,402-934-2064,infusionbrewing.com,0,0


In [168]:
testDF[testDF['BreweryName'] == 'Nebraska Brewing Company']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
68,7136,Nebraska Brewing Company,Brewpub,https://beermapping.com/location/7136,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,7474 Towne Center Pkwy. Suite 101,Papillion,NE,68046,United States,(402) 934-7100,nebraskabrewingco.com,91.666675,8
69,18889,Nebraska Brewing Company,Brewery,https://beermapping.com/location/18889,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,6950 S 108th St,La Vista,NE,68128,United States,,nebraskabrewingco.com,0,0


In [170]:
testDF[testDF['BreweryName'] == 'Thunderhead Brewing Company']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
94,1326,Thunderhead Brewing Company,Brewpub,https://beermapping.com/location/1326,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,18 East 21st Street,Kearney,NE,68847,United States,(308) 237-1558,thunderheadbrewing.com/,0,0
95,16272,Thunderhead Brewing Company,Brewery,https://beermapping.com/location/16272,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,201 Avenue F,Axtell,NE,68924,United States,(308) 237-1558,thunderheadbrewing.com,0,0


In [169]:
testDF[testDF['BreweryName'] == 'Zipline Brewing Company']

,BreweryID,BreweryName,Type,ReviewLink,ProxyLink,Map,StreetAddress,City,State,Zip,Country,PhoneNum,Website,Rating,ImageCount
103,16511,Zipline Brewing Company,Brewery,https://beermapping.com/location/16511,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,"2100 Magnum Circle, Ste 1",Lincoln,NE,68522,United States,(402) 450-9804,ziplinebrewing.com,0,0
104,21370,Zipline Brewing Company,Brewery,https://beermapping.com/location/21370,http://beermapping.com/maps/proxymaps.php?loci...,http://beermapping.com/maps/blogproxy.php?loci...,721 N 14th St,Omaha,NE,68102,United States,,http://ziplinebrewing.com/,0,0
